In [1]:
import pandas as pd
import numpy as np
import altair as alt
import requests

In [2]:
pallete = {
    "shadow": "rgba(24, 42, 56, 0.1)",
    "nominal_1": "#179fdb",
    "nominal_2": "#e6224b",
    "nominal_3": "#f4c245",
    "nominal_4": "#122b39",
    "nominal_5": "#eb5c2e",
    "nominal_6": "#36b7b4",
    "Other_3":"#d6d4d4",
    #"Deemphasize_Discrete": "rgb(173, 195, 215)",
    "Deemphasize_Discrete": "rgba(24, 42, 56)",
    "Deemphasize_Other": "rgba(93, 115, 102, 1)",
    "Deemphasize_Continuous": "rgba(24, 42, 56, 0.2)",
    "accent": "#179fdb",
    "domain": "#122b39",
    "bar": {
        "accent_1": "#122b39",
        "other" : "#a8c0de",
        "accent_2" : "#eb5c2e"
    }
}

pallete["GBR"] = pallete["nominal_1"]
pallete["USA"] = pallete["nominal_2"]
pallete["DEU"] = pallete["nominal_3"]
pallete["FRA"] = pallete["nominal_4"]
pallete["Other_1"] = pallete["nominal_5"]
pallete["Other_2"] = pallete["nominal_6"]

pallete["UK"] = pallete["GBR"]
pallete["United Kingdom"] = pallete["GBR"]
pallete["United States"] = pallete["USA"]
pallete["Germany"] = pallete["DEU"]
pallete["France"] = pallete["FRA"]

def dark():
    return {"config": {
        "background": "#122B39",
        "font": "Circular Std",
        "title": {
            "color": "#b4c8d8",
            "fontSize": 14,
            "fontWeight": 400,
        },
        "mark": {
            "line": {
                "interpolate": "monotone",
                "color": "#36B7B4"
            }
        },
        "view": {
            "stroke": "transparent",
            "width": 400,
            "height": 300
        },
        "range": {
            "category": ["#36B7B4", "#E6224B", "#F4C245", "#0063AF", "#00A767", "#179FDB", "#EB5C2E"],
            "diverging": ["#E6224B", "#E54753", "#C9C9C9", "#179FDB", "#122B39"],
            "heatmap": ["#C9C9C9", "#179FDB", "#0063AF", "#122B39"],
            "ordinal": ["#00A767", "#36B7B4", "#179FDB", "#0063AF", "#243B5A"]
        },
        "axisX": {
            "domainColor": "#b4c8d8",
            "domainOpacity": 0.5,
            "grid": False,
            "labelAngle": 0,
            "labelColor": "#b4c8d8",
                "labelOpacity": 0.7,
                "orient": "bottom",
                "tickColor": "#b4c8d8",
                "tickCount": 10,
                "tickOpacity": 0.5,
                "title": "",
                "titleAlign": "center",
                "titleAnchor": "middle",
                "titleColor": "#b4c8d8",
                "titleFontSize": 12,
                "titleOpacity": 0.8,
                "titleX": 207,
                "titleY": -15
        },
        "axisY": {
            "domainColor": "#b4c8d8",
            "domainOpacity": 0.5,
            "format": ".0f",
            "gridColor": "#b4c8d8",
            "gridDash": [
                1,
                5
            ],
            "gridOpacity": 0.5,
            "labelColor": "#b4c8d8",
            "labelOpacity": 0.7,
            "labelPadding": 5,
            "tickColor": "#b4c8d8",
            "tickCount": 8,
            "tickOpacity": 0.5,
            "ticks": False,
            "title": "FAO price index",
            "titleAlign": "left",
            "titleAngle": 0,
            "titleBaseline": "bottom",
            "titleColor": "#b4c8d8",
            "titleFontSize": 12,
            "titleOpacity": 0.8,
            "titleX": 0,
            "titleY": -7

        }
    }}


alt.themes.register('dark', dark)

<function __main__.dark()>

In [3]:
req = requests.get("https://api.ons.gov.uk/timeseries/KAC3/dataset/LMS/data")

In [80]:

data = req.json()["months"]
df = pd.DataFrame(data)
df['value'] = df['value'].astype(float)/100
df['date'] = pd.to_datetime(df['date'], format="%Y %b")
df = df.reset_index()
df = df[df["date"] > "2020-01-01"]

alt.themes.enable('dark')

base = alt.Chart(df).encode(
    x=alt.X('date:T', axis=alt.Axis(format='%Y', values=[
        {"year": 2020, "month": 1, "date": 1},
        {"year": 2021, "month": 1, "date": 1},
        {"year": 2022, "month": 1, "date": 1},
        {"year": 2023, "month": 1, "date": 1},
    ])),
    y=alt.Y('value:Q', axis=alt.Axis(format='%')),
)

line = base.mark_line(color="#36B7B4")

labels_0 = base.transform_filter(
    "datum.index == 271"
).mark_text(
    align='left',
    baseline='middle',
    dy=-5,
    color="#36B7B4",
    text="7.2%",
    dx=3,
    #dy=alt.expr("datum.index == 271 ? 5  : 0"),
).encode(
    text=alt.value("October")
)

labels_1 = base.transform_filter(
    "datum.index == 271"
).mark_text(
    align='left',
    baseline='middle',
    color="#36B7B4",
    dy=5,
    text="7.2%",
    dx=3,
    #dy=alt.expr("datum.index == 271 ? 5  : 0"),
).encode(
    text=alt.value("7.2%")
)

zero_rule = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color="#b4c8d8", opacity=0.5).encode(y='y')

chart = line + labels_0 + labels_1 + zero_rule

#chart.save("gfcf_country.json")
#chart.save("gfcf_country.png", scale_factor=2.0)

chart = chart.properties(
    width=300,
    height=200,
        title={
        "text": ["UK Pay Growth Falls"],
        "subtitle": ["Year on Year Three Month Average Pay Growth (%):", "Source: ONS"],
                             "subtitleColor": "#b4c8d8",
        "fontSize": 12,
        "color": "#b4c8d8",
        "anchor": "start",
        "font": "Circular Std",
        "subtitleFont": "Circular Std",
        "dx": 25,
        "subtitleFontSize": 10,
    },

)

chart.save("wage_growth.json")
chart.save("wage_growth.png", scale_factor=2.0)

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

In [4]:
df

,date,value
0,1971-02-01,3.8
1,1971-03-01,3.9
2,1971-04-01,4.0
3,1971-05-01,4.1
4,1971-06-01,4.1
...,...,...
624,2023-02-01,3.9
625,2023-03-01,3.8
626,2023-04-01,4.0
627,2023-05-01,4.2
